In [5]:
from shared_code.utility.spark.set_environ import *
from shared_code.utility.schemas.spark_table_schema import image_table_schema

set_azure_env()

spark = get_session("add-thumbnails")

In [6]:
spark_df = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images.parquet").collect(), schema=image_table_schema)

print(f"Total Number Of Records From Pre-Processing {spark_df.count()}")

display(spark_df.limit(10).toPandas())

Total Number Of Records From Pre-Processing 6501


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
5,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
6,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
7,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
8,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
9,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False


In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import PIL.Image
import logging

def resize_image(path: str):
	img = PIL.Image.open(path)
	try:
		copied_image = img.copy()
		i = copied_image.resize((512, 512))
		return i
	finally:
		img.close()


@udf(returnType=StringType())
def get_resized_image(big_image_path, subreddit, image_name):
	try:
		expected_path = os.path.join("D:", os.sep, "data", "images", subreddit, "thumbnail", image_name)
		if os.path.exists(expected_path):
			return expected_path
		else:
			logging.info(f"Resizing image {big_image_path} to {expected_path}...")
			r = resize_image(big_image_path)
			r.save(expected_path)
			r.close()
			return expected_path
	except Exception as e:
		logging.error(f"Error resizing image {big_image_path}...")
		return ""

In [16]:
result = spark_df \
	.withColumn("small_image", get_resized_image("image", "subreddit", "image_name")).collect()

print(f"Total Number of Records Updated To Write: {len(result)}")

writeable_df = spark.createDataFrame(result, schema=image_table_schema)

display(writeable_df.toPandas())

writeable_df.write.parquet("D:\\data\\processed\\reddit_images_processed.parquet", mode="overwrite")

Total Number of Records Updated To Write: 6501


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496,training,zp0v3q,D:\data\images\SFWNextDoorGirls\uo6pi0dzsp6a1.jpg,"Happy Sunday, neighbors 😇",zp0v3q,The__virtuosa,https://i.redd.it/uo6pi0dzsp6a1.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/zp0v3q/happy_sund...,9ebe24fe107600b5c7ed11a703278ccc,SFWNextDoorGirls,A girl in a pink dress is holding a baby doll.,True,uo6pi0dzsp6a1.jpg,a woman in a white shirt and black pants posin...,D:\data\images\SFWNextDoorGirls\thumbnail\uo6p...,False
6497,training,zp18ej,D:\data\images\SFWNextDoorGirls\itex51v7wp6a1.jpg,This cold front that just came in has this sou...,zp18ej,sexxymilf1030,https://i.redd.it/itex51v7wp6a1.jpg,Swimwear,/r/SFWNextDoorGirls/comments/zp18ej/this_cold_...,6f4fbbd32b7cd98f2229a6cd9d45a116,SFWNextDoorGirls,A woman in a bathing suit is sitting on a beac...,False,itex51v7wp6a1.jpg,NaN,,False
6498,training,zp1cgf,D:\data\images\SFWNextDoorGirls\3bzbjskdfo6a1.jpg,Showing off my abs just for you guys 😘,zp1cgf,Kattyax,https://i.redd.it/3bzbjskdfo6a1.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/zp1cgf/showing_of...,c2128c839fbf07b59a77ce50fd7ef679,SFWNextDoorGirls,A woman in a pink shirt and blue jeans is stan...,True,3bzbjskdfo6a1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\3bzb...,False
6499,training,zp1kyc,D:\data\images\SFWNextDoorGirls\p7d6cdx9zp6a1.jpg,sfw Feeling the holiday spirit,zp1kyc,Just_peachy_xo,https://i.redd.it/p7d6cdx9zp6a1.jpg,Just Feelin' Cute 💕,/r/SFWNextDoorGirls/comments/zp1kyc/sfw_feelin...,a3341e0f7406d7189827fae9c65d0ec0,SFWNextDoorGirls,A woman in a red dress is standing in front of...,False,p7d6cdx9zp6a1.jpg,NaN,,False


In [17]:
new_frame = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images_processed.parquet").collect(), schema=image_table_schema)

display(new_frame.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
5,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
6,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
7,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
8,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
9,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
